<a href="https://colab.research.google.com/github/KangHwan-Cha/MyStudy/blob/main/TensorProject/Category4A__NLP-training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
import json
import tensorflow as tf
import numpy as np
import urllib

In [143]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

In [144]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
urllib.request.urlretrieve(url, 'sarcasm.json')

('sarcasm.json', <http.client.HTTPMessage at 0x7f60d1c690a0>)

In [145]:
import os
for a in os.walk(os.getcwd()):
  print(a)

('/content', ['.config', 'sample_data'], ['sarcasm.json'])
('/content/.config', ['logs', 'configurations'], ['gce', 'active_config', '.last_survey_prompt.yaml', '.last_update_check.json', 'config_sentinel', '.last_opt_in_prompt.yaml'])
('/content/.config/logs', ['2023.01.20'], [])
('/content/.config/logs/2023.01.20', [], ['14.34.30.476454.log', '14.33.57.412394.log', '14.34.57.156862.log', '14.34.22.572544.log', '14.34.56.325690.log', '14.33.33.386047.log'])
('/content/.config/configurations', [], ['config_default'])
('/content/sample_data', [], ['README.md', 'anscombe.json', 'california_housing_train.csv', 'mnist_test.csv', 'mnist_train_small.csv', 'california_housing_test.csv'])


In [146]:
with open('sarcasm.json') as f:
    datas = json.load(f)
type(datas), datas[:3]

(list,
 [{'article_link': 'https://www.huffingtonpost.com/entry/versace-black-code_us_5861fbefe4b0de3a08f600d5',
   'headline': "former versace store clerk sues over secret 'black code' for minority shoppers",
   'is_sarcastic': 0},
  {'article_link': 'https://www.huffingtonpost.com/entry/roseanne-revival-review_us_5ab3a497e4b054d118e04365',
   'headline': "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
   'is_sarcastic': 0},
  {'article_link': 'https://local.theonion.com/mom-starting-to-fear-son-s-web-series-closest-thing-she-1819576697',
   'headline': "mom starting to fear son's web series closest thing she will have to grandchild",
   'is_sarcastic': 1}])

In [147]:
sentences = []
labels = []
for data in datas:
    sentences.append(data['headline'])
    labels.append(data['is_sarcastic'])

In [148]:
sentences[:5]

["former versace store clerk sues over secret 'black code' for minority shoppers",
 "the 'roseanne' revival catches up to our thorny political mood, for better and worse",
 "mom starting to fear son's web series closest thing she will have to grandchild",
 'boehner just wants wife to listen, not come up with alternative debt-reduction ideas',
 'j.k. rowling wishes snape happy birthday in the most magical way']

In [149]:
labels[:5]

[0, 0, 1, 1, 0]

In [150]:
training_size = 20000

In [151]:
train_sentences = sentences[:training_size]
train_labels = labels[:training_size]
validation_sentences = sentences[training_size:]
validation_labels = labels[training_size:]

In [152]:
vocab_size = 1000
oov_tok = "<OOV>"

In [153]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)

In [154]:
tokenizer.fit_on_texts(train_sentences)

In [155]:
for key, value in tokenizer.word_index.items():
    print(f'{key}   \t====>\t {value}')
    if value == 7:
        break

print('')
print(f'단어 사전의 개수: {len(tokenizer.word_index)}')

<OOV>   	====>	 1
to   	====>	 2
of   	====>	 3
the   	====>	 4
in   	====>	 5
for   	====>	 6
a   	====>	 7

단어 사전의 개수: 25637


In [156]:
word_index = tokenizer.word_index

In [157]:
print(word_index['trump'])
print(word_index['hello'])
print(word_index['<OOV>'])

13
11724
1


In [158]:
train_sequences = tokenizer.texts_to_sequences(train_sentences)
validation_sequences = tokenizer.texts_to_sequences(validation_sentences)

In [159]:
import pprint
pprint.pprint(train_sequences[:5])
pprint.pprint(validation_sequences[:5])

[[328, 1, 799, 1, 1, 47, 389, 1, 1, 6, 1, 1],
 [4, 1, 1, 1, 23, 2, 161, 1, 390, 1, 6, 251, 9, 889],
 [153, 890, 2, 891, 1, 1, 595, 1, 221, 133, 36, 45, 2, 1],
 [1, 38, 213, 382, 2, 1, 29, 288, 23, 10, 1, 1, 1, 958],
 [715, 672, 1, 1, 1, 662, 553, 5, 4, 92, 1, 90]]
[[1, 1, 1, 1, 30, 1, 1, 5, 519, 109],
 [202, 1, 8, 31, 1, 1, 2, 854, 773],
 [18, 380, 191, 2, 915, 76, 8, 4, 1],
 [1, 1, 299, 337, 3, 1, 1],
 [162, 1, 1, 6, 1, 1, 348, 1]]


In [160]:
train_sentences[4]

'j.k. rowling wishes snape happy birthday in the most magical way'

In [161]:
word_index['j'], word_index['k'], word_index['rowling'], word_index['wishes'], word_index['snape'], word_index['happy']

(715, 672, 5652, 1043, 8865, 662)

In [162]:
train_sequences[4]

[715, 672, 1, 1, 1, 662, 553, 5, 4, 92, 1, 90]

### 시퀀스의 길이 맞춰주기

In [163]:
max_length = 120

trunc_type='post'

padding_type='post'

In [164]:
train_padded = pad_sequences(train_sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)
validation_padded = pad_sequences(validation_sequences, maxlen=max_length, truncating=trunc_type, padding=padding_type)


In [165]:
type(train_padded), train_padded.shape

(numpy.ndarray, (20000, 120))

In [166]:
train_padded[0]

array([328,   1, 799,   1,   1,  47, 389,   1,   1,   6,   1,   1,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0], dtype=int32)

### label 값을 numpy array로 변환

In [167]:
train_labels = np.array(train_labels)
validation_labels = np.array(validation_labels)

### Embedding Layer

In [168]:
embedding_dim = 16

### 모델 정의(Sequential(

In [169]:
model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    Bidirectional(LSTM(64, return_sequences=True)),
    Bidirectional(LSTM(64)),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [170]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 120, 16)           16000     
                                                                 
 bidirectional_9 (Bidirectio  (None, 120, 128)         41472     
 nal)                                                            
                                                                 
 bidirectional_10 (Bidirecti  (None, 128)              98816     
 onal)                                                           
                                                                 
 dense_10 (Dense)            (None, 32)                4128      
                                                                 
 dense_11 (Dense)            (None, 16)                528       
                                                                 
 dense_12 (Dense)            (None, 1)                

In [171]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [172]:
checkpoint_path = 'my_checkpoint.ckpt'
checkpoint = ModelCheckpoint(filepath=checkpoint_path,
                             save_weights_only=True,
                             save_best_only=True,
                             monitor='val_loss',
                             verbose=1)

### 학습(fit)

In [173]:
epochs = 10

In [174]:
model.fit(train_padded, train_labels,
            validation_data=(validation_padded, validation_labels),
            callbacks=[checkpoint],
            epochs=epochs)

Epoch 1/10
624/625 [============================>.] - ETA: 0s - loss: 0.4548 - acc: 0.7705
Epoch 1: val_loss improved from inf to 0.39908, saving model to my_checkpoint.ckpt
625/625 [==============================] - 19s 25ms/step - loss: 0.4549 - acc: 0.7704 - val_loss: 0.3991 - val_acc: 0.8077
Epoch 2/10
625/625 [==============================] - ETA: 0s - loss: 0.3534 - acc: 0.8383
Epoch 2: val_loss improved from 0.39908 to 0.37091, saving model to my_checkpoint.ckpt
625/625 [==============================] - 14s 22ms/step - loss: 0.3534 - acc: 0.8383 - val_loss: 0.3709 - val_acc: 0.8323
Epoch 3/10
624/625 [============================>.] - ETA: 0s - loss: 0.3266 - acc: 0.8558
Epoch 3: val_loss improved from 0.37091 to 0.36935, saving model to my_checkpoint.ckpt
625/625 [==============================] - 14s 22ms/step - loss: 0.3268 - acc: 0.8555 - val_loss: 0.3693 - val_acc: 0.8334
Epoch 4/10
624/625 [============================>.] - ETA: 0s - loss: 0.3085 - acc: 0.8633
Epoch 4: v